In [ ]:
!pip install -q peft transformers datasets

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator, get_linear_schedule_with_warmup
from peft import get_peft_config, get_peft_model, PromptTuningInit, PromptTuningConfig, TaskType, PeftType
import torch
from datasets import load_dataset
import os
from torch.utils.data import DataLoader
from tqdm import tqdm

device = "cuda"
model_name_or_path = "tiiuae/falcon-7b-instruct"
tokenizer_name_or_path = "tiiuae/falcon-7b-instruct"
peft_config = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM,
    prompt_tuning_init=PromptTuningInit.TEXT,
    num_virtual_tokens=8,
    prompt_tuning_init_text="Classify if the tweet is a complaint or not:",
    tokenizer_name_or_path=model_name_or_path,
)

dataset_name = "twitter_complaints"
checkpoint_name = f"{dataset_name}_{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}_v1.pt".replace(
    "/", "_"
)
text_column = "Dialogue"
label_column = "Character"
max_length = 64
lr = 3e-2
num_epochs = 50
batch_size = 8

ValueError: Unable to compare versions for numpy>=1.17: need=1.17 found=None. This is unusual. Consider reinstalling numpy.

In [ ]:
dataset = load_dataset('csv', data_files='data/all_labeled_lines.csv')
dataset = dataset['train'].train_test_split(test_size=0.2)

In [ ]:
classes = {"spongebob": 1, "not spongebob": 0}
dataset = dataset.map(
    lambda x: {"Label": [classes[label] for label in x["Character"]]},
    batched=True,
    num_proc=1,
)
dataset["train"][0]

Map:   0%|          | 0/33641 [00:00<?, ? examples/s]

Map:   0%|          | 0/8411 [00:00<?, ? examples/s]

{'Dialogue': 'Huh?', 'Character': 'not spongebob', 'Label': 0}

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
target_max_length = max([len(tokenizer(class_label)["input_ids"]) for class_label in classes])
print(target_max_length)

5


In [ ]:
def preprocess_function(examples):
    batch_size = len(examples[text_column])
    inputs = [f"{text_column} : {x} Label : " for x in examples[text_column]]
    targets = [str(x) for x in examples[label_column]]
    model_inputs = tokenizer(inputs)
    labels = tokenizer(targets)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i] + [tokenizer.pad_token_id]
        # print(i, sample_input_ids, label_input_ids)
        model_inputs["input_ids"][i] = sample_input_ids + label_input_ids
        labels["input_ids"][i] = [-100] * len(sample_input_ids) + label_input_ids
        model_inputs["attention_mask"][i] = [1] * len(model_inputs["input_ids"][i])
    # print(model_inputs)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i]
        model_inputs["input_ids"][i] = [tokenizer.pad_token_id] * (
            max_length - len(sample_input_ids)
        ) + sample_input_ids
        model_inputs["attention_mask"][i] = [0] * (max_length - len(sample_input_ids)) + model_inputs[
            "attention_mask"
        ][i]
        labels["input_ids"][i] = [-100] * (max_length - len(sample_input_ids)) + label_input_ids
        model_inputs["input_ids"][i] = torch.tensor(model_inputs["input_ids"][i][:max_length])
        model_inputs["attention_mask"][i] = torch.tensor(model_inputs["attention_mask"][i][:max_length])
        labels["input_ids"][i] = torch.tensor(labels["input_ids"][i][:max_length])
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
processed_datasets = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=dataset["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

Running tokenizer on dataset:   0%|          | 0/33641 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/8411 [00:00<?, ? examples/s]

In [ ]:
train_dataset = processed_datasets["train"]
eval_dataset = processed_datasets["test"]


train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True
)
eval_dataloader = DataLoader(eval_dataset, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
model = get_peft_model(model, peft_config)
print(model.print_trainable_parameters())

config.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

trainable params: 8,192 || all params: 559,222,784 || trainable%: 0.0014648902430985358
None


In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader) * num_epochs),
)

In [ ]:
model = model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.detach().float()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    eval_loss = 0
    eval_preds = []
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.detach().float()
        eval_preds.extend(
            tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), skip_special_tokens=True)
        )

    eval_epoch_loss = eval_loss / len(eval_dataloader)
    eval_ppl = torch.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(train_dataloader)
    train_ppl = torch.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

100%|██████████| 1052/1052 [03:42<00:00,  4.73it/s]


epoch=0: train_ppl=tensor(1.2702, device='cuda:0') train_epoch_loss=tensor(0.2392, device='cuda:0') eval_ppl=tensor(1.1175, device='cuda:0') eval_epoch_loss=tensor(0.1111, device='cuda:0')


100%|██████████| 1052/1052 [03:42<00:00,  4.73it/s]


epoch=1: train_ppl=tensor(1.1159, device='cuda:0') train_epoch_loss=tensor(0.1097, device='cuda:0') eval_ppl=tensor(1.1563, device='cuda:0') eval_epoch_loss=tensor(0.1452, device='cuda:0')


100%|██████████| 1052/1052 [03:42<00:00,  4.73it/s]


...

100%|██████████| 1052/1052 [03:42<00:00,  4.72it/s]


epoch=4: train_ppl=tensor(1.1066, device='cuda:0') train_epoch_loss=tensor(0.1013, device='cuda:0') eval_ppl=tensor(1.1045, device='cuda:0') eval_epoch_loss=tensor(0.0994, device='cuda:0')


100%|██████████| 1052/1052 [03:42<00:00,  4.73it/s]


epoch=5: train_ppl=tensor(1.1070, device='cuda:0') train_epoch_loss=tensor(0.1017, device='cuda:0') eval_ppl=tensor(1.1045, device='cuda:0') eval_epoch_loss=tensor(0.0994, device='cuda:0')


 33%|███▎      | 1384/4206 [09:47<19:57,  2.36it/s]